# ¿Qué es *web scraping*?

El término web scraping se refiere a la descarga de datos de la web en un formato legible. 

# ¿Cómo se hace web scraping?

Para extraer datos de una web, se puede utilizar código de Python o recurrir a otras opciones, como utilizar una API o herramientas específicas para estas tareas. 

***Nota***: una API es una interfaz de programación de aplicaciones, que permite la comunicación entre dos aplicaciones de software a través de un conjunto de reglas. 

Algunos sitios web, como Twitter, facilitan directamente una API para poder acceder a sus datos. El problema es que la mayoría de los sitios web no tienen API o, incluso cuando sí que tienen, los datos que permiten obtener no son los que nosotros necesitamos. En estos casos, crear un rastreador web con Python es una buena solución.

# Conceptos previos de páginas webs

Cuando visitamos una página web, nuestro navegador manda una solicitud a un servidor web para obtener archivos de dicho servidor. El servidor devuelve archivos que le indican a nuestro navegador cómo debe mostrar la página. Básicamente, estos archivos pueden ser:
- De tipo HTML, que contienen el contenido principal de una página.
- De tipo CSS, que agregan estilos de colores y formatos a la página.
- De tipo JS (Javascript), que agregan interactividad a las páginas web.
- De imagen, como JPG y PNG, que permiten que las páginas web muestren imágenes.

En principio, nos interesa el contenido principal de la página web, por lo que nuestro objetivo es extraer datos del archivo HTML.

# Ejemplo práctico

In [1]:
# pip install requests
# pip install beautifulsoup4
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"   # Direccion de la pagina con la data que queremos

In [10]:
html_content = requests.get(url).text  # Obtenemos la pagina en formato html

In [6]:
soup = BeautifulSoup(html_content)  # creamos una instancia de la libreria que importamos utilizando el contenido de la pagina (el texto html)

In [8]:
#print(soup.prettify())   # usamos un metodo para lo que recien instanciamos, esto nos hace ver un poco mas ordenado el texto

In [9]:
soup.title.text # ejemplo de conseguir el titulo de la pagina en este caso .text quita las etiquetas de los lados

'List of S&P 500 companies - Wikipedia'

In [11]:
tabla_500 = soup.find("table", attrs={"id": "constituents"})  # attrs una manera de identificar la tabla en la pagina

## Encabezados de la tabla

In [30]:
encabezados = tabla_500.tbody
linea_encabezado = encabezados.tr
linea_encabezado

<tr>
<th><a href="/wiki/Ticker_symbol" title="Ticker symbol">Symbol</a>
</th>
<th>Security</th>
<th><a href="/wiki/Global_Industry_Classification_Standard" title="Global Industry Classification Standard">GICS</a> Sector</th>
<th>GICS Sub-Industry</th>
<th>Headquarters Location</th>
<th>Date added</th>
<th><a href="/wiki/Central_Index_Key" title="Central Index Key">CIK</a></th>
<th>Founded
</th></tr>

In [34]:
#lista para guardar los nombres de las columnas
columnas = []


for th in linea_encabezado.find_all("th"):  #Busco este valor en mis datos ya que ahi se encuentran los nombres de los encabezados
    columnas.append(th.text.replace("\n", "").strip()) # Limpio el nombre y lo agrego a la lista
    
columnas

['Symbol',
 'Security',
 'GICS Sector',
 'GICS Sub-Industry',
 'Headquarters Location',
 'Date added',
 'CIK',
 'Founded']

In [35]:
datos_tabla = tabla_500.tbody
datos_tabla

<tbody><tr>
<th><a href="/wiki/Ticker_symbol" title="Ticker symbol">Symbol</a>
</th>
<th>Security</th>
<th><a href="/wiki/Global_Industry_Classification_Standard" title="Global Industry Classification Standard">GICS</a> Sector</th>
<th>GICS Sub-Industry</th>
<th>Headquarters Location</th>
<th>Date added</th>
<th><a href="/wiki/Central_Index_Key" title="Central Index Key">CIK</a></th>
<th>Founded
</th></tr>
<tr>
<td><a class="external text" href="https://www.nyse.com/quote/XNYS:MMM" rel="nofollow">MMM</a>
</td>
<td><a href="/wiki/3M" title="3M">3M</a></td>
<td>Industrials</td>
<td>Industrial Conglomerates</td>
<td><a href="/wiki/Saint_Paul,_Minnesota" title="Saint Paul, Minnesota">Saint Paul, Minnesota</a></td>
<td>1957-03-04</td>
<td>0000066740</td>
<td>1902
</td></tr>
<tr>
<td><a class="external text" href="https://www.nyse.com/quote/XNYS:AOS" rel="nofollow">AOS</a>
</td>
<td><a href="/wiki/A._O._Smith" title="A. O. Smith">A. O. Smith</a></td>
<td>Industrials</td>
<td>Building Product

## filas de la tabla

In [37]:
len(datos_tabla.find_all("tr"))  # vemos la cantidad de registros

504

In [41]:
filas = [] # lista que contendra cada fila

for linea in datos_tabla.find_all("tr"): # Cada que pasemos por un valor tr creamos una lista
    fila = []
    for td in linea.find_all("td"):   # Cada que pasemos por valores td agregamos el texto a la lista que creamos un paso antes
        fila.append(td.text.replace("\n","").strip())
    filas.append(fila)   # Esa lista recien creada la agregamos a la lista llamada filas

filas.remove([]) # Los encabezados no tenian td por lo que no recopilo info pero si genero una lista vacia
filas

[['MMM',
  '3M',
  'Industrials',
  'Industrial Conglomerates',
  'Saint Paul, Minnesota',
  '1957-03-04',
  '0000066740',
  '1902'],
 ['AOS',
  'A. O. Smith',
  'Industrials',
  'Building Products',
  'Milwaukee, Wisconsin',
  '2017-07-26',
  '0000091142',
  '1916'],
 ['ABT',
  'Abbott',
  'Health Care',
  'Health Care Equipment',
  'North Chicago, Illinois',
  '1957-03-04',
  '0000001800',
  '1888'],
 ['ABBV',
  'AbbVie',
  'Health Care',
  'Pharmaceuticals',
  'North Chicago, Illinois',
  '2012-12-31',
  '0001551152',
  '2013 (1888)'],
 ['ACN',
  'Accenture',
  'Information Technology',
  'IT Consulting & Other Services',
  'Dublin, Ireland',
  '2011-07-06',
  '0001467373',
  '1989'],
 ['ATVI',
  'Activision Blizzard',
  'Communication Services',
  'Interactive Home Entertainment',
  'Santa Monica, California',
  '2015-08-31',
  '0000718877',
  '2008'],
 ['ADM',
  'ADM',
  'Consumer Staples',
  'Agricultural Products & Services',
  'Chicago, Illinois',
  '1957-03-04',
  '0000007084'

In [42]:
df = pd.DataFrame(data=filas, columns=columnas)

In [43]:
df.head()

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,0000066740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,0000091142,1916
2,ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,0000001800,1888
3,ABBV,AbbVie,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,0001551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,0001467373,1989


In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 503 entries, 0 to 502
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Symbol                 503 non-null    object
 1   Security               503 non-null    object
 2   GICS Sector            503 non-null    object
 3   GICS Sub-Industry      503 non-null    object
 4   Headquarters Location  503 non-null    object
 5   Date added             503 non-null    object
 6   CIK                    503 non-null    object
 7   Founded                503 non-null    object
dtypes: object(8)
memory usage: 31.6+ KB
